# Load the Pre-trained ProteinMPNN Model

First, load the pre-trained ProteinMPNN model, ensuring you have the correct model architecture and class definitions.

In [2]:
import torch
from protein_mpnn_utils import ProteinMPNN

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
#v_48_010=version with 48 edges 0.10A noise
model_name = "v_48_030" #@param ["v_48_002", "v_48_010", "v_48_020", "v_48_030"]


backbone_noise=0.00               # Standard deviation of Gaussian noise to add to backbone atoms

path_to_model_weights='ProteinMPNN/vanilla_model_weights'
hidden_dim = 128
num_layers = 3
model_folder_path = path_to_model_weights
if model_folder_path[-1] != '/':
    model_folder_path = model_folder_path + '/'
checkpoint_path = model_folder_path + f'{model_name}.pt'

checkpoint = torch.load(checkpoint_path, map_location=device)
# Initialize the model
proteinmpnn_model = ProteinMPNN(num_letters=21, node_features=hidden_dim, edge_features=hidden_dim, hidden_dim=hidden_dim, num_encoder_layers=num_layers, num_decoder_layers=num_layers, augment_eps=backbone_noise, k_neighbors=checkpoint['num_edges'])
proteinmpnn_model.to(device)
proteinmpnn_model.load_state_dict(checkpoint['model_state_dict'])
proteinmpnn_model.eval()
print("Model loaded")
# Load the pre-trained weights

Model loaded


# Extract Encoder Weights
Since the encoder is composed of multiple EncLayer instances within a ModuleList, you can directly access this list and its constituent layers. Each layer in the list is an instance of EncLayer, and you can work with them individually or as a whole.

In [2]:
# Access the ModuleList of encoder layers
encoder_layers = proteinmpnn_model.encoder_layers

# You can iterate over each layer if needed
for idx, enc_layer in enumerate(encoder_layers):
    # Extract the state_dict of each encoder layer if needed
    enc_layer_weights = enc_layer.state_dict()
    # You can save or directly use these weights as per your requirement
    # For example, saving weights with a layer-specific filename
    torch.save(enc_layer_weights, f"EncoderWeights/encoder_layer_{idx}_weights.pt")


# Use Encoder Weights in Your New Model
When initializing the encoder layers in your new model, you can load these extracted weights into each corresponding layer. Ensure that the architecture of the encoder layers in your new model matches the architecture of the layers from which you extracted the weights.

In [6]:
from protein_mpnn_utils import EncLayer  # Ensure this is the same EncLayer used in ProteinMPNN

# Initialize your new encoder layers (make sure it matches the original architecture)
num_encoder_layers = num_layers
new_encoder_layers = torch.nn.ModuleList([
    EncLayer(hidden_dim, hidden_dim*2)
    for _ in range(num_encoder_layers)
])

# Load weights into your new encoder layers
for idx, enc_layer in enumerate(new_encoder_layers):
    # Load the previously saved weights for each layer
    enc_layer_weights = torch.load(f"EncoderWeights/encoder_layer_{idx}_weights.pt")
    enc_layer.load_state_dict(enc_layer_weights)

print(new_encoder_layers)

ModuleList(
  (0-2): 3 x EncLayer(
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
    (dropout3): Dropout(p=0.1, inplace=False)
    (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (norm3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (W1): Linear(in_features=384, out_features=128, bias=True)
    (W2): Linear(in_features=128, out_features=128, bias=True)
    (W3): Linear(in_features=128, out_features=128, bias=True)
    (W11): Linear(in_features=384, out_features=128, bias=True)
    (W12): Linear(in_features=128, out_features=128, bias=True)
    (W13): Linear(in_features=128, out_features=128, bias=True)
    (act): GELU(approximate='none')
    (dense): PositionWiseFeedForward(
      (W_in): Linear(in_features=128, out_features=512, bias=True)
      (W_out): Linear(in_features=512, out_features=128, bias=True)
      (act): GELU(approximate='none'

# Considerations
Ensure the new encoder layers have the same configuration (hidden_dim, dropout, etc.) as the original layers to guarantee compatibility.
When extracting and using weights like this, be mindful of any preprocessing or postprocessing steps that might be required for the inputs or outputs of these layers, as such steps might be integral to how the encoder layers function within the context of the full ProteinMPNN model.
If your use case involves modifying the encoder architecture or integrating it into a significantly different model, careful adjustments might be needed to ensure that the extracted weights remain effective in the new context.



